In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
!pip install umap-learn
import umap
!pip install optuna
import optuna
import xgboost as xgb
import functools
import lightgbm as lgb
import time
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

In [2]:
from cycler import cycler
'''
raw_light_palette = [
    (0, 122, 255), # Blue
    (255, 149, 0), # Orange
    (52, 199, 89), # Green
    (255, 59, 48), # Red
    (175, 82, 222),# Purple
    (255, 45, 85), # Pink
    (88, 86, 214), # Indigo
    (90, 200, 250),# Teal
    (255, 204, 0)  # Yellow
]

raw_dark_palette = [
    (10, 132, 255), # Blue
    (255, 159, 10), # Orange
    (48, 209, 88),  # Green
    (255, 69, 58),  # Red
    (191, 90, 242), # Purple
    (94, 92, 230),  # Indigo
    (255, 55, 95),  # Pink
    (100, 210, 255),# Teal
    (255, 214, 10)  # Yellow
]

raw_gray_light_palette = [
    (142, 142, 147),# Gray
    (174, 174, 178),# Gray (2)
    (199, 199, 204),# Gray (3)
    (209, 209, 214),# Gray (4)
    (229, 229, 234),# Gray (5)
    (242, 242, 247),# Gray (6)
]

raw_gray_dark_palette = [
    (142, 142, 147),# Gray
    (99, 99, 102),  # Gray (2)
    (72, 72, 74),   # Gray (3)
    (58, 58, 60),   # Gray (4)
    (44, 44, 46),   # Gray (5)
    (28, 28, 39),   # Gray (6)
]


light_palette = np.array(raw_light_palette)/255
dark_palette = np.array(raw_dark_palette)/255
gray_light_palette = np.array(raw_gray_light_palette)/255
gray_dark_palette = np.array(raw_gray_dark_palette)/255

mpl.rcParams['axes.prop_cycle'] = cycler('color',dark_palette)
mpl.rcParams['figure.facecolor']  = gray_dark_palette[-2]
mpl.rcParams['figure.edgecolor']  = gray_dark_palette[-2]
mpl.rcParams['axes.facecolor'] =  gray_dark_palette[-2]

white_color = gray_light_palette[-2]
mpl.rcParams['text.color'] = white_color
mpl.rcParams['axes.labelcolor'] = white_color
mpl.rcParams['axes.edgecolor'] = white_color
mpl.rcParams['xtick.color'] = white_color
mpl.rcParams['ytick.color'] = white_color

mpl.rcParams['figure.dpi'] = 200

mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

sns.palplot(dark_palette)
'''

"\nraw_light_palette = [\n    (0, 122, 255), # Blue\n    (255, 149, 0), # Orange\n    (52, 199, 89), # Green\n    (255, 59, 48), # Red\n    (175, 82, 222),# Purple\n    (255, 45, 85), # Pink\n    (88, 86, 214), # Indigo\n    (90, 200, 250),# Teal\n    (255, 204, 0)  # Yellow\n]\n\nraw_dark_palette = [\n    (10, 132, 255), # Blue\n    (255, 159, 10), # Orange\n    (48, 209, 88),  # Green\n    (255, 69, 58),  # Red\n    (191, 90, 242), # Purple\n    (94, 92, 230),  # Indigo\n    (255, 55, 95),  # Pink\n    (100, 210, 255),# Teal\n    (255, 214, 10)  # Yellow\n]\n\nraw_gray_light_palette = [\n    (142, 142, 147),# Gray\n    (174, 174, 178),# Gray (2)\n    (199, 199, 204),# Gray (3)\n    (209, 209, 214),# Gray (4)\n    (229, 229, 234),# Gray (5)\n    (242, 242, 247),# Gray (6)\n]\n\nraw_gray_dark_palette = [\n    (142, 142, 147),# Gray\n    (99, 99, 102),  # Gray (2)\n    (72, 72, 74),   # Gray (3)\n    (58, 58, 60),   # Gray (4)\n    (44, 44, 46),   # Gray (5)\n    (28, 28, 39),   # Gray 

In [3]:
api_token = {"username":"watsons","key":"dc7da47ca9aa5e696b65f97c16fd627b"}
import json
import zipfile
import os
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")
 
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
 
if not os.path.exists("/kaggle"):
    os.makedirs("/kaggle")
os.chdir('/kaggle')
!kaggle competitions download -c tabular-playground-series-may-2021
!unzip \*.zip  && rm *.zip

  0% 0.00/851k [00:00<?, ?B/s]
100% 851k/851k [00:00<00:00, 45.5MB/s]
  0% 0.00/1.72M [00:00<?, ?B/s]
100% 1.72M/1.72M [00:00<00:00, 112MB/s]
  0% 0.00/128k [00:00<?, ?B/s]
100% 128k/128k [00:00<00:00, 78.8MB/s]
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               

Archive:  sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   

3 archives were successfully processed.


In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [5]:
train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,21,0,0,0,0,0,0,Class_2
1,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
2,2,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,0,0,1,0,5,2,1,1,0,1,0,2,0,0,0,0,19,2,0,1,7,3,0,0,0,1,0,0,0,0,13,2,0,Class_1
3,3,0,0,0,0,0,0,0,3,0,0,0,0,0,1,7,11,1,0,3,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,Class_4
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,Class_2


In [6]:
print(train.shape)
print(test.shape)

(100000, 52)
(50000, 51)


In [7]:
train.isnull().any().sum()

0

In [8]:
test.isnull().any().sum()

0

In [9]:
train['target'].value_counts()

Class_2    57497
Class_3    21420
Class_4    12593
Class_1     8490
Name: target, dtype: int64

In [10]:
'''
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
y = np.array([train[f'feature_{i}'].nunique() for i in range(50)])
y2 = np.array([test[f'feature_{i}'].nunique() for i in range(50)])
comp = y-y2
ax.bar(range(50), y2, alpha=0.7, color=gray_dark_palette[0], label='Test Dataset')
# Thanks to @rahulchauhan3j to fix typo
ax.bar(range(50),  comp*(comp>0), bottom=y2, color=dark_palette[2], alpha=0.7, label='Train > Test')
ax.bar(range(50), comp*(comp<0), bottom=y2-comp*(comp<0), color=dark_palette[3], alpha=0.7, label='Train < Test')
ax.set_yticks(range(0, 80, 5))
ax.margins(0.02)
ax.grid(axis='y', linestyle='--', zorder=5)
ax.set_title('# of Features Unique Values (Train/Test)', loc='left', fontweight='bold')
ax.legend()
plt.show()
'''

"\nfig, ax = plt.subplots(1, 1, figsize=(12, 6))\ny = np.array([train[f'feature_{i}'].nunique() for i in range(50)])\ny2 = np.array([test[f'feature_{i}'].nunique() for i in range(50)])\ncomp = y-y2\nax.bar(range(50), y2, alpha=0.7, color=gray_dark_palette[0], label='Test Dataset')\n# Thanks to @rahulchauhan3j to fix typo\nax.bar(range(50),  comp*(comp>0), bottom=y2, color=dark_palette[2], alpha=0.7, label='Train > Test')\nax.bar(range(50), comp*(comp<0), bottom=y2-comp*(comp<0), color=dark_palette[3], alpha=0.7, label='Train < Test')\nax.set_yticks(range(0, 80, 5))\nax.margins(0.02)\nax.grid(axis='y', linestyle='--', zorder=5)\nax.set_title('# of Features Unique Values (Train/Test)', loc='left', fontweight='bold')\nax.legend()\nplt.show()\n"

In [11]:
'''
fig, axes = plt.subplots(13, 4, figsize=(10, 16))
target_order = sorted(train['target'].unique())

for idx, ax in zip(range(50), axes.flatten()):
    cnt = train[f'feature_{idx}'].value_counts().sort_index()
    sns.kdeplot(x=f'feature_{idx}', 
    hue='target', hue_order=target_order,
    data=train,
    alpha=0.5, 
    linewidth=0.6, fill=True,
    legend=False,
    ax=ax)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.spines['left'].set_visible(False)
    cnt = len(train[f'feature_{idx}'].unique())
    ax.set_title(f'Feature_{idx}({cnt})', loc='right', weight='bold', fontsize=11)

axes.flatten()[-1].axis('off')    
axes.flatten()[-2].axis('off')

fig.tight_layout()
plt.show()
'''

"\nfig, axes = plt.subplots(13, 4, figsize=(10, 16))\ntarget_order = sorted(train['target'].unique())\n\nfor idx, ax in zip(range(50), axes.flatten()):\n    cnt = train[f'feature_{idx}'].value_counts().sort_index()\n    sns.kdeplot(x=f'feature_{idx}', \n    hue='target', hue_order=target_order,\n    data=train,\n    alpha=0.5, \n    linewidth=0.6, fill=True,\n    legend=False,\n    ax=ax)\n    \n    ax.set_xticks([])\n    ax.set_yticks([])\n    ax.set_xlabel('')\n    ax.set_ylabel('')\n    ax.spines['left'].set_visible(False)\n    cnt = len(train[f'feature_{idx}'].unique())\n    ax.set_title(f'Feature_{idx}({cnt})', loc='right', weight='bold', fontsize=11)\n\naxes.flatten()[-1].axis('off')    \naxes.flatten()[-2].axis('off')\n\nfig.tight_layout()\nplt.show()\n"

In [12]:
def balanced_subsample(train,size=None):
    subsample = []
    if size is None:
        n_smp = train['target'].value_counts().min()
    else:
        n_smp = int(size / len(train['target'].value_counts().index))
    for label in train['target'].value_counts().index:
        samples = train[train['target'] == label].index.values
        index_range = range(samples.shape[0])
        indexes = np.random.choice(index_range, size=n_smp, replace=False)
        subsample += samples[indexes].tolist()
    return subsample

In [13]:
train.set_index('id',drop=True,inplace=True)
y=train.target
one_hot_y=pd.get_dummies(y)
train.drop('target',axis=1,inplace=True)
test.set_index('id',drop=True,inplace=True)
train_test=pd.concat([train,test])

In [14]:
'''
for i,col in enumerate(train.columns):
    replace=list(set(test.iloc[:,i]).difference(set(train.iloc[:,i])))
    for num in replace:
        test.iloc[:,i].replace(num,0,inplace=True)
'''

'\nfor i,col in enumerate(train.columns):\n    replace=list(set(test.iloc[:,i]).difference(set(train.iloc[:,i])))\n    for num in replace:\n        test.iloc[:,i].replace(num,0,inplace=True)\n'

In [15]:
class variables:
    def _init_(self,data):
        self.data=data
    def statistics(self,data,combined):
        combined['sum']=data.sum(1)
        combined['avg']=data.mean(1)
        combined['max']=data.max(1)
        combined['min']=data.min(1)
        combined['median']=data.median(1)
        combined['std']=data.std(1)
        combined['count0']=data.isin([0]).sum(1)
        return combined

In [16]:
class encoding:
    def _init_(self,train_dup):
        self.train=train_dup
    def target_encoding(self,train_dup,y,kf):
        columns=train_dup.columns
        for c in columns:
            data_tmp=pd.DataFrame({c:train_dup[c],'target':y})
            tmp=np.repeat(np.nan,train_dup.shape[0])
            for i,(tr_idx,va_idx) in enumerate(kf.split(train_dup)):
                target_mean=data_tmp.iloc[tr_idx].groupby(c)['target'].mean()
                tmp[va_idx]=train_dup[c].iloc[va_idx].map(target_mean)
            train_dup[c]=tmp
        train_dup.fillna(0,inplace=True)
        return train_dup
'''
kf=KFold(n_splits=5, random_state=23, shuffle=True)
train_for=train
train_dup=train
logsave=pd.DataFrame(np.zeros((60,4)))
logsave.columns=['col','cat','down','score']
count=0
t1 = time.time()

collist=['Class_1','Class_2','Class_3','Class_4']
catlist=['ordinal','standard','one_hot','target_enc','nothing']
downlist=['lda','umap','nothing']
for col in range(3,4):
    ytrain=one_hot_y.iloc[:,col]
    for cat in range(2,5):
        if(cat==0):
            ordinal=preprocessing.OrdinalEncoder().fit(train_for)
            xtrain=pd.DataFrame(ordinal.transform(train_for),columns=train_for.columns)
        elif(cat==1):
            xtrain=preprocessing.StandardScaler().fit_transform(train_for)
        elif(cat==2):
            one_hot=preprocessing.OneHotEncoder().fit(train_for)
            xtrain=pd.DataFrame(one_hot.transform(train_for).toarray())
            xtrain.columns=one_hot.get_feature_names(train_for.columns)
        elif(cat==3):
            xtrain=encoding().target_encoding(train_dup,one_hot_y.iloc[:,col],kf)
        else:
            xtrain=train
        for down in range(3):
            xtrain=pd.DataFrame(xtrain)
            if(down==0):
                clf = LinearDiscriminantAnalysis()
                xtrain_down=clf.fit_transform(xtrain,one_hot_y.iloc[:,col])
                xtrain_down=pd.DataFrame(xtrain_down)
                xtrain_down=pd.concat([xtrain_down,xtrain],axis=1)
            elif(down==1):
                um=umap.UMAP()
                xtrain_down =um.fit_transform(xtrain)
                xtrain_down=pd.DataFrame(xtrain_down)
                xtrain_down=pd.concat([xtrain_down,xtrain],axis=1)
            else:
                xtrain_down=xtrain

            xtrain_down=variables().statistics(train_for,xtrain_down)

            lgbc=lgb.LGBMClassifier()
            scores = cross_val_score(lgbc,xtrain_down,one_hot_y.iloc[:,col],scoring='neg_log_loss',cv=kf)
            score=-round(scores.mean(),3)

            logsave.iloc[count,:]=[collist[col],catlist[cat],downlist[down],score]
            count+=1
            print(count)
            t2 = time.time()   
            print('time elapsed: ' + str(round(t2-t1, 2)) + ' seconds')
            print(list(logsave.iloc[count,:]))
'''        

"\nkf=KFold(n_splits=5, random_state=23, shuffle=True)\ntrain_for=train\ntrain_dup=train\nlogsave=pd.DataFrame(np.zeros((60,4)))\nlogsave.columns=['col','cat','down','score']\ncount=0\nt1 = time.time()\n\ncollist=['Class_1','Class_2','Class_3','Class_4']\ncatlist=['ordinal','standard','one_hot','target_enc','nothing']\ndownlist=['lda','umap','nothing']\nfor col in range(3,4):\n    ytrain=one_hot_y.iloc[:,col]\n    for cat in range(2,5):\n        if(cat==0):\n            ordinal=preprocessing.OrdinalEncoder().fit(train_for)\n            xtrain=pd.DataFrame(ordinal.transform(train_for),columns=train_for.columns)\n        elif(cat==1):\n            xtrain=preprocessing.StandardScaler().fit_transform(train_for)\n        elif(cat==2):\n            one_hot=preprocessing.OneHotEncoder().fit(train_for)\n            xtrain=pd.DataFrame(one_hot.transform(train_for).toarray())\n            xtrain.columns=one_hot.get_feature_names(train_for.columns)\n        elif(cat==3):\n            xtrain=encod

In [25]:
def lb_opt(xtrain,ytrain,trial):

    bagging_freq =  trial.suggest_int('bagging_freq',1,10),
    min_data_in_leaf =  trial.suggest_int('min_data_in_leaf',2,100),
    max_depth = trial.suggest_int('max_depth',1,20),
    learning_rate = trial.suggest_loguniform('learning_rate',0.001,0.1),
    num_leaves = trial.suggest_int('num_leaves',2,70),
    num_threads = trial.suggest_int('num_threads',1,10),
    min_sum_hessian_in_leaf = trial.suggest_int('min_sum_hessian_in_leaf',1,10),
    n_estimators = trial.suggest_int('n_estimators',10,500),
    min_child_samples = trial.suggest_int('n_estimators',10,30),
    reg_lambda =trial.suggest_float('reg_lambda',0.,1.)
    reg_alpha = trial.suggest_float('reg_alpha',0.,1.)
    subsample = trial.suggest_float('subsample',0.5,1.)
    colsample_bytree = trial.suggest_float('colsample_bytree',0.5,1.)
    lightgbm_tuna = lgb.LGBMClassifier(
        random_state = 0,
        verbosity = 1,
        bagging_seed = 0,
        metric = 'binary_logloss',
        min_child_samples = min_child_samples,
        reg_alpha = reg_alpha,
        reg_lambda =reg_lambda,
        bagging_freq = bagging_freq ,
        min_data_in_leaf = min_data_in_leaf,
        max_depth = max_depth,
        learning_rate = learning_rate,
        num_leaves = num_leaves,
        num_threads = num_threads,
        min_sum_hessian_in_leaf = min_sum_hessian_in_leaf,
        colsample_bytree=colsample_bytree,
        subsample = subsample
    )
    kf=KFold(n_splits=5, random_state=23, shuffle=True)
    scores = cross_val_score(lightgbm_tuna,xtrain,ytrain,scoring='neg_log_loss',cv=kf)
    score = -round(scores.mean(),3)
    return score

In [18]:
'''
optuna.logging.disable_default_handler()
lb_study = optuna.create_study()
lb_study.optimize(functools.partial(lb_opt,xtrain,ytrain),n_trials = 100)
print(lb_study.best_params)
'''

'\noptuna.logging.disable_default_handler()\nlb_study = optuna.create_study()\nlb_study.optimize(functools.partial(lb_opt,xtrain,ytrain),n_trials = 100)\nprint(lb_study.best_params)\n'

In [19]:
train_test.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,21,0,0,0,0,0,0
1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,0,0,1,0,5,2,1,1,0,1,0,2,0,0,0,0,19,2,0,1,7,3,0,0,0,1,0,0,0,0,13,2,0
3,0,0,0,0,0,0,0,3,0,0,0,0,0,1,7,11,1,0,3,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [20]:
ordinal=preprocessing.OrdinalEncoder().fit(train_test)
x=pd.DataFrame(ordinal.transform(train_test),columns=train.columns)
x=variables().statistics(train_test,x)
ytrain=preprocessing.LabelEncoder().fit_transform(y)
xtrain=x[:100000]
xtest=x[100000:]

In [21]:
x_std=train_test
x_std.columns=[name+'_std' for name in train_test.columns]
for col in range(50):
  for col2 in range(col+1,51):
    plus=str(col)+'+'+str(col2)
    x_std[plus]=x_std.iloc[:,col]+x_std.iloc[:,col2]

    '''    
    mult=str(col)+'x'+str(col2)
    divi=str(col)+'/'+str(col2)
    x_std[mult]=x_std.iloc[:,col]*x_std.iloc[:,col2]
    x_std[divi]=x_std.iloc[:,col]/x_std.iloc[:,col2]
    '''

In [22]:
x_std=pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(x_std),columns=x_std.columns)
xtrain_std=x_std[:100000]
xtest_std=x_std[100000:]
xtrain_std=pd.concat([xtrain,xtrain_std],axis=1)
xtest_std=pd.concat([xtest,xtest_std],axis=1)

In [23]:
xtrain_std.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,41+47,41+48,41+49,41+50,42+43,42+44,42+45,42+46,42+47,42+48,42+49,42+50,43+44,43+45,43+46,43+47,43+48,43+49,43+50,44+45,44+46,44+47,44+48,44+49,44+50,45+46,45+47,45+48,45+49,45+50,46+47,46+48,46+49,46+50,47+48,47+49,47+50,48+49,48+50,49+50
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,0.0,1.0,1.0,0.0,0.0,5.0,4.0,...,0.000000,0.000000,0.0,0.0,0.403509,0.051282,0.044444,0.031746,0.033898,0.037037,0.042553,0.021739,0.636364,0.403846,0.446809,0.525,0.437500,0.583333,0.381818,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,0.0,0.0,1.0,0.0,0.0,5.0,4.0,...,0.000000,0.000000,0.0,0.0,0.035088,0.051282,0.044444,0.031746,0.033898,0.037037,0.042553,0.021739,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,2.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,2.0,3.0,2.0,19.0,2.0,1.0,1.0,7.0,8.0,4.0,...,0.382353,0.039216,0.0,0.0,0.052632,0.076923,0.066667,0.047619,0.271186,0.092593,0.063830,0.043478,0.000000,0.000000,0.000000,0.325,0.041667,0.000000,0.000000,0.0,0.0,0.481481,0.041667,0.0,0.0,0.0,0.382353,0.039216,0.0,0.0,0.282609,0.04,0.0,0.0,0.326087,0.393939,0.317073,0.040816,0.033898,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,11.0,1.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,3.0,2.0,0.0,1.0,1.0,0.0,0.0,9.0,4.0,...,0.000000,0.019608,0.0,0.0,0.035088,0.051282,0.044444,0.031746,0.033898,0.055556,0.042553,0.021739,0.000000,0.000000,0.000000,0.000,0.020833,0.000000,0.000000,0.0,0.0,0.000000,0.020833,0.0,0.0,0.0,0.000000,0.019608,0.0,0.0,0.000000,0.02,0.0,0.0,0.021739,0.000000,0.000000,0.020408,0.016949,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,9.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,2.0,0.0,0.0,1.0,0.0,0.0,6.0,4.0,...,0.000000,0.019608,0.0,0.0,0.035088,0.051282,0.044444,0.031746,0.033898,0.055556,0.042553,0.021739,0.000000,0.000000,0.000000,0.000,0.020833,0.000000,0.000000,0.0,0.0,0.000000,0.020833,0.0,0.0,0.0,0.000000,0.019608,0.0,0.0,0.000000,0.02,0.0,0.0,0.021739,0.000000,0.000000,0.020408,0.016949,0.0


In [ ]:
ytrain=pd.DataFrame(ytrain,columns=['target'])
zip=pd.concat([xtrain,ytrain],axis=1)
train_bal=zip.iloc[balanced_subsample(zip)]
y_bal=train_bal.target
train_bal.drop('target',axis=1,inplace=True)

In [31]:
out=pd.DataFrame()
for i in range(4):  
  lgbm = lgb.LGBMClassifier() 
  lgbm.fit(xtrain_std,one_hot_y.iloc[:,i])
  booster = lgbm.booster_
  importance = booster.feature_importance(importance_type='split')
  feature_name = booster.feature_name()
  feature_importance = pd.DataFrame({'feature_name':feature_name,'importance':importance} )
  feature_importance.sort_values(by='importance',ascending=False,inplace=True)
  xtrain_select=xtrain_std[feature_importance.iloc[:100,0]]
  optuna.logging.disable_default_handler()
  lb_study = optuna.create_study()
  lb_study.optimize(functools.partial(lb_opt,xtrain_select,one_hot_y.iloc[:,i]),n_trials = 50)
  lgbm = lgb.LGBMClassifier(**lb_study.best_params) 
  lgbm.fit(xtrain_select,one_hot_y.iloc[:,i])
  tmp=pd.DataFrame(lgbm.predict_proba(xtest_std[feature_importance.iloc[:100,0]]))
  out=pd.concat([out,tmp.iloc[:,1]],axis=1)

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

In [ ]:
'''
out.columns=one_hot_y.columns
out2.columns=one_hot_y.columns
xtrain_plusout=pd.concat([xtrain_std,out2],axis=1)
xtest_plusout=pd.concat([xtest_std,out],axis=1)
lgbm = lgb.LGBMClassifier()
lgbm.fit(xtrain_plusout,ytrain)
out3=pd.DataFrame(lgbm.predict_proba(xtest_plusout))
out3.head()
'''

In [ ]:
out=pd.DataFrame()
out2=pd.DataFrame()
for i in range(4):  
  lgbm = lgb.LGBMClassifier()
  kf=KFold(n_splits=5, random_state=23, shuffle=True)
  lgbm.fit(xtrain,one_hot_y.iloc[:,i])
  tmp2=pd.DataFrame(lgbm.predict_proba(xtrain))
  tmp=pd.DataFrame(lgbm.predict_proba(xtest))
  out2=pd.concat([out2,tmp2.iloc[:,1]],axis=1)
  out=pd.concat([out,tmp.iloc[:,1]],axis=1)

In [ ]:
out.columns=one_hot_y.columns
out2.columns=one_hot_y.columns
xtrain_plusout=pd.concat([xtrain,out2],axis=1)
xtest_plusout=pd.concat([xtest,out],axis=1)
lgbm = lgb.LGBMClassifier()
lgbm.fit(xtrain_plusout,ytrain)
out3=pd.DataFrame(lgbm.predict_proba(xtest_plusout))
out3.head()

In [32]:
for i in range(4):
  sample_submission.iloc[:,i+1]=out.iloc[:,i]
sample_submission

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.087046,0.626582,0.149010,0.119671
1,100001,0.083141,0.841707,0.099929,0.083179
2,100002,0.086924,0.636390,0.176524,0.103132
3,100003,0.088786,0.572574,0.378350,0.088330
4,100004,0.082272,0.594972,0.182368,0.125203
...,...,...,...,...,...
49995,149995,0.084692,0.692845,0.165548,0.078793
49996,149996,0.084441,0.686398,0.123526,0.132235
49997,149997,0.082940,0.529355,0.187321,0.177003
49998,149998,0.085111,0.642656,0.161992,0.142338


In [33]:
sample_submission.to_csv('sub_0526.csv',index=False)

In [34]:
!kaggle competitions submit -c tabular-playground-series-may-2021 -f sub_0526.csv -m "0527"

100% 4.06M/4.06M [00:00<00:00, 10.2MB/s]
Successfully submitted to Tabular Playground Series - May 2021